# Librerías y configuraciones

In [71]:
# Arrays para manipular datos
import numpy as np
import pandas as pd

# Gráficos
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12, 8)
import seaborn as sns
sns.set(style="darkgrid", font_scale=1.5)

# Ciencia de datos, machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Carga de datos

In [ ]:
# https://drive.google.com/file/d/1sWLmuazrRKlb1_c88soFRqQYF0wXCxgT/view?usp=sharing
data = pd.read_csv("./wine.csv")
data.head()

# Escalado



In [73]:
scaler = StandardScaler()
scaler.fit(data)
data_escalada = scaler.transform(data)

# PCA exploratorio



In [ ]:
# instancio la clase
pca = PCA(n_components=2)
# calculo PCA
pca.fit(data_escalada)
# transformo los datos originales al plano PCA
pca_data = pca.transform(data_escalada)

# cálculo varianza explicada
vars = pca.explained_variance_ratio_
var1 = round(100*vars[0],2)
var2 = round(100*vars[1],2)

# gráfico
plt.scatter(x=pca_data[:,0], y=pca_data[:,1], lw=2)
plt.xlabel("Componente principal 1 ({}%)".format(var1))
plt.ylabel("Componente principal 2 ({}%)".format(var2))
plt.title("Análisis por componentes principales")
plt.show()

# Clustering (agrupamiento) por K-means

In [ ]:
# Elijo la cantidad de clusters
cant_clusters = 3
# instancio la clase
kmeans = KMeans(n_clusters=cant_clusters)#, random_state=42)
# entreno al modelo
kmeans.fit(data_escalada)
# que me prediga el cluster para cada dato
y_kmeans = pd.Series(kmeans.predict(data_escalada))

# que me diga los centroides
centroides = kmeans.cluster_centers_
# transformo los centroides al plano PCA para poderlos graficar
centroides_pca = pca.transform(centroides)

# grafico en el plano PCA datos, clusters y centroides
plt.scatter(pca_data[:,0],pca_data[:,1], c=kmeans.labels_, cmap="Paired")
plt.scatter(centroides_pca[:,0],centroides_pca[:, 1], c="red", s=200)
plt.xlabel("Componente principal 1 ({}%)".format(var1))
plt.ylabel("Componente principal 2 ({}%)".format(var2))
plt.title("Visualización de clusters por PCA")
plt.show()

# Exploración de las características de los clusters

In [ ]:
# Agrega una nueva variable con el cluster al que pertenece cada observación
data["Cluster"] = y_kmeans

# Calculo la media de cada variable para cada cluster
data.groupby(["Cluster"]).mean()

In [ ]:
# densidades para cada variable, y scatter plots dos a dos, identificando cada cluster.
sns.pairplot(data, hue="Cluster", palette=['green','orange','brown'])
plt.show()

# Si hago K-means sobre los datos **sin** escalar, ¿las características de los clusters cambian?

In [ ]:
# Elijo la cantidad de clusters
cant_clusters = 3
# instancio la clase
kmeans = KMeans(n_clusters=cant_clusters)#, random_state=42)
# entreno al modelo en datos SIN escalar
kmeans.fit(data) 
# que me prediga el cluster para cada dato SIN escalar
y_kmeans = pd.Series(kmeans.predict(data))


# Agrega una nueva variable con el cluster al que pertenece cada observación
data["Cluster"] = y_kmeans
# Calculo la media de cada variable para cada cluster
data.groupby(["Cluster"]).mean()

In [ ]:
# densidades para cada variable, y scatter plots dos a dos, identificando cada cluster.
sns.pairplot(data, hue="Cluster", palette=['green','orange','brown'])
plt.show()

# CORRER y comparar contra el anterior.

# ¿Y se puede visualizar por PCA con los datos sin escalar?

In [ ]:
# instancio la clase
pca = PCA(n_components=2)
# calculo PCA sobre datos SIN escalar
pca.fit(data)
# transformo los datos originales al plano PCA
pca_data = pca.transform(data)

# calculo varianza explicada
vars = pca.explained_variance_ratio_
var1 = round(100*vars[0],2)
var2 = round(100*vars[1],2)

# que me diga los centroides
centroides = kmeans.cluster_centers_
# transformo los centroides al plano PCA para poderlos graficar
centroides_pca = pca.transform(centroides)

# grafico en el plano PCA datos, clusters y centroides
plt.scatter(pca_data[:,0],pca_data[:,1], c=kmeans.labels_, cmap="Paired")
plt.scatter(centroides_pca[:,0],centroides_pca[:, 1], c="red", s=200)
plt.xlabel("Componente principal 1 ({}%)".format(var1))
plt.ylabel("Componente principal 2 ({}%)".format(var2))
plt.title("Visualización de clusters por PCA (datos sin escalar)")
plt.show()


# ¿Y cómo averiguar las posibles cantidades de clusters?

## Una forma es por el Silhouette score

In [ ]:
# Silhouette score: va entre -1 y 1 (1 es lo mejor).
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html
# https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient

silhouette_scores = []
rango_clusters = range(2,21)

for i in rango_clusters:
    # para cada valor de i, calculo kmeans y silhouette
    k_means = KMeans(n_clusters=i)
    k_means.fit(data_escalada)
    
    sil_score = silhouette_score(data_escalada, labels=k_means.labels_)
    silhouette_scores.append(sil_score)

plt.plot(rango_clusters,silhouette_scores)
plt.xticks(rango_clusters)
plt.xlabel('Cantidad de clusters')
plt.ylabel('Score Silhouette')
plt.title('Los máximos locales son los candidatos según Silhouette')
plt.show()